# 付喪神ジェネレータ 香るブラック君 vs 午後の紅茶さん


RTX-4090 実行: 2024/9/14

In [1]:
!nvidia-smi

Sun Sep 15 08:58:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090 L...    On | 00000000:01:00.0  On |                  N/A |
| N/A   47C    P8                5W /  55W|   5350MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%cd ../
%cd SadTalker
# %ls

/home/atsu/llamaindex
/home/atsu/llamaindex/SadTalker


/home/atsu/anaconda3/envs/llamaindex/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# # Whisperのinstal --> 一度だけ実行
# !pip install git+https://github.com/openai/whisper.git

In [4]:
from copy import deepcopy
import uuid # 追加
import re
import os
import glob
import random
import time
import gradio as gr
import io
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import openai
from huggingface_hub import login
import whisper

使用した画像を削除してしまうので以下のコードをコメントアウトした。  
shutil.move(source_image, input_dir) 

In [6]:
# SadTalker用
# from src.gradio_demo2 import SadTalker
from inference6 import make_video # face_enhancer, background_enhancer 対応

In [7]:
# VOICEVOX 用
from pathlib import Path
import voicevox_core
from voicevox_core import AccelerationMode, AudioQuery, VoicevoxCore
from playsound import playsound
from pydub import AudioSegment
# from IPython.display import Audio

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [8]:
# 基本パラメータ
model_id = "tokyotech-llm/Swallow-MS-7b-instruct-v0.1"
# ブラック君
peft_name_black = "../付喪神ジェネレータ/QLoRA_models/black_mistral"
# 紅茶さん
peft_name_tea = "../付喪神ジェネレータ/QLoRA_models/tea_mistral"

In [9]:
# OpenAI API ログイン
from openai import OpenAI
with open('../API_key/OpenAI_API_key.txt', 'r') as f:
    key = f.read()
os.environ["OPENAI_API_KEY"] = key
openai.api_key = key
client = OpenAI()

# Gradioを用いたWebアプリ作成

In [10]:
# 音声からテキストへの変換関数（新バージョン）
# https://platform.openai.com/docs/guides/speech-to-text/quickstart
def speech_to_text_API(input_audio):
    audio_file = open(input_audio, "rb")
    transcription = client.audio.transcriptions.create(
        model="whisper-1", 
        file=audio_file,
    )
    return transcription.text

In [11]:
# VOICEVOX によるテキストから音声変換
def VOICEVOX(text, out='output.wav', SPEAKER_ID=2):
    open_jtalk_dict_dir = './open_jtalk_dic_utf_8-1.11'
    acceleration_mode = AccelerationMode.AUTO
    core = VoicevoxCore(
        acceleration_mode=acceleration_mode, open_jtalk_dict_dir=open_jtalk_dict_dir
    )
    core.load_model(SPEAKER_ID)
    audio_query = core.audio_query(text, SPEAKER_ID)
    wav = core.synthesis(audio_query, SPEAKER_ID)
    out_byte = Path(out) # 追加
    out_byte.write_bytes(wav)
    return out

In [12]:
# モデルとトークナイザの準備
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# モデルの設定
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    # token=token, # HuggingFaceにログインしておけば不要
    quantization_config=bnb_config, # 量子化
    device_map='auto',
    torch_dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
)
# tokenizerの設定
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    padding_side="right",
    add_eos_token=True
)
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
%%time
# ファインチューニングモデルの作成
base_model_2 = deepcopy(base_model)
model_black = PeftModel.from_pretrained(base_model, peft_name_black)
model_tea = PeftModel.from_pretrained(base_model_2, peft_name_tea)

CPU times: user 7.47 s, sys: 642 ms, total: 8.11 s
Wall time: 4.01 s


In [14]:
# 文章生成関数: black君
def generate_black(prompt, max_tokens=256, temperature=1, top_p=0.9, top_k=30):

    prompt = f"""system prompt:あなたは缶コーヒーを飲むユーザーと対話するAIです。大阪弁で対話してください。短く返答してください。
    user prompt: {prompt}"""
    
    messages = [
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_black.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model_black.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model_black.device),
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)
    return response_text


In [15]:
# 文章生成関数: teaさん
def generate_tea(prompt, max_tokens=256, temperature=1, top_p=0.9, top_k=30):

    prompt = f"""system prompt:あなたは午後の紅茶を飲むユーザーと対話するAIです。お嬢さま口調で対話してください。短く返答してください。
    user prompt: {prompt}"""
    
    messages = [
        {"role": "user", "content": prompt},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model_tea.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model_tea.generate(
        input_ids,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model_tea.device),
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_text = tokenizer.decode(response, skip_special_tokens=True)
    return response_text


In [16]:
def overall_response(input_mode, input_text, input_audio, max_tokens):
    
    # GPUメモリをリセット
    torch.cuda.empty_cache()
    
    if input_mode == "Text":
        input_text = input_text
    else:
        input_text = speech_to_text_API(input_audio)

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 応答
    code_regex = re.compile('[!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠！｀ 　＋￥％？ξﾟ⊿abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789]')

    seed = random.randint(0,1)
    print("===============================")
    print(seed)
    if seed==0:
        # black君
        response_text_black = code_regex.sub('', generate_black(input_text, max_tokens)).replace("\n","")
        response_text_black = response_text_black.replace("ですが","やけど").replace("ありますが","あるんやけど").replace("あり。","ありますねん。").\
                                      replace("しているのやねん。","してるんねん。").replace("わい、思いますねん。","").replace("わい", "俺")
        candidates = ["で、どう思う？", "そうなんや、ええな", "ほんま？", "まじで？", "オチは？"]
        sample = random.choice(candidates)
        response_text_black = response_text_black.replace("知らんけど", sample)
        # teaさん
        response_text_tea = code_regex.sub('', generate_tea(response_text_black, max_tokens)).replace("\n","")
        print("response text black: " + response_text_black)
        print("response text tea: " + response_text_tea)
        print("===============================")
        
    else:
        # teaさん
        response_text_tea = code_regex.sub('', generate_tea(input_text, max_tokens)).replace("\n","")
        # black君
        response_text_black = code_regex.sub('', generate_black(response_text_tea, max_tokens)).replace("\n","")
        response_text_black = response_text_black.replace("ですが","やけど").replace("ありますが","あるんやけど").replace("あり。","ありますねん。").\
                                      replace("しているのやねん。","してるんねん。").replace("わい、思いますねん。","").replace("わい", "俺")
        candidates = ["で、どう思う？", "そうなんや、ええな", "ほんま？", "まじで？", "オチは？"]
        sample = random.choice(candidates)
        response_text_black = response_text_black.replace("知らんけど", sample)
        print("response text tea: " + response_text_tea)
        print("response text black: " + response_text_black)
        print("===============================")
        
    # GPUメモリをリセット
    torch.cuda.empty_cache()

    # 返答を音声に
    out_black = "../付喪神ジェネレータ/results/output_black.wav"
    out_tea = "../付喪神ジェネレータ/results/output_tea.wav"
    response_audio_black = VOICEVOX(text=response_text_black, out=out_black, SPEAKER_ID=13) #青山龍星
    response_audio_tea = VOICEVOX(text=response_text_tea, out=out_tea, SPEAKER_ID=17) #九州そらセクシー
    
    print("Sound generation finished.") # ターミナルに表示

    # 画像パスを引数に変更
    image_path_black = "../付喪神ジェネレータ/agents/香るブラック君.png"
    image_path_tea = "../付喪神ジェネレータ/agents/午後の紅茶さん.png"

    # 生成ビデオ保存フォルダ
    result_dir = '../付喪神ジェネレータ/results'
    
    responce_video_black = make_video(image_path=image_path_black,
                                audio_path=response_audio_black,
                                size=256,
                                preprocess="extcrop",
                                enhancer=None,
                                background_enhancer=None,
                                result_dir=result_dir,
                                )

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    responce_video_tea = make_video(image_path=image_path_tea,
                                audio_path=response_audio_tea,
                                size=256,
                                preprocess="extcrop",
                                enhancer=None,
                                background_enhancer=None,
                                result_dir=result_dir,
                                )
    
    print("Video generation finished.") # ターミナルに表示

    # GPUメモリをリセット
    torch.cuda.empty_cache()

    if seed==0:
        responce_video_1 = responce_video_black
        responce_video_2 = responce_video_tea
    else:
        responce_video_1 = responce_video_tea
        responce_video_2 = responce_video_black
        
    print(seed)
    print("responce_video_1", responce_video_1)
    print("responce_video_2", responce_video_2)

    return responce_video_1, responce_video_2

In [17]:
# Blockの定義
# https://note.com/npaka/n/n2a5112208b8d

# CSSを使用してボタンのスタイルをカスタマイズ
css = """
#custom-button {
    background: #ee7800; /* ボタンの背景色 */
    color: white; /* ボタンのテキスト色 */
    border: none;
    text-align: center;
    font-size: 20px;
    cursor: pointer;
}
"""

with gr.Blocks(css=css) as app:

    # Markdown
    gr.Markdown("""# 超未来の付喪神ジェネレータ　香るブラック君 vs 午後のミルクティさん""")

    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            input_mode = gr.Radio(["Text","Voice"], label="Input mode", value="Text")
            input_text = gr.Textbox(placeholder="Please write the input.", label="input text",)
            record = gr.Audio(sources=["microphone"], label="Input audio", type="filepath")
            clear = gr.ClearButton(components=[record])
            max_tokens = gr.Dropdown([32, 64, 128, 256, 512, 1024, 2048, 4096, 8192], label="max tokens", value=32)
            btn = gr.Button("Response", elem_id="custom-button") #ボタンにIDを指定            
        with gr.Column(scale=1, min_width=300):
            response_video_1 = gr.Video()
        with gr.Column(scale=1, min_width=300):
            response_video_2 = gr.Video()
            # イベントリスナー
            btn.click(fn=overall_response,
                      inputs=[input_mode, input_text, record, max_tokens],                   
                      outputs=[response_video_1, response_video_2]
                      )

In [ ]:
# Webアプリを起動
app.launch(share=True, debug=True) # share=Trueで一時的に公開される

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://365f6f2a883230b9e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


0
response text black: 午後の紅茶が好きやねん。オチは？。
response text tea: ワタクシ思いますの、午後の紅茶が好きです、オチは。
Sound generation finished.
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|███████████████████████████████████| 34/34 [00:33<00:00,  1.01it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 241) to (256, 256) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The generated video is named ../付喪神ジェネレータ/results/2024_09_15_09.02.35/香るブラック君##output_black.mp4
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|███████████████████████████████████| 83/83 [00:53<00:00,  1.57it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

The generated video is named ../付喪神ジェネレータ/results/2024_09_15_09.35.11/午後の紅茶さん##output_tea.mp4
Video generation finished.
1
responce_video_1 ../付喪神ジェネレータ/results/2024_09_15_09.35.11.mp4
responce_video_2 ../付喪神ジェネレータ/results/2024_09_15_09.34.59.mp4
0
response text black: コーヒーは、エネルギーと覚醒を与えてくれるわ、そうなんや、ええな。
response text tea: ワタクシ思いますの、午後の紅茶は、カフェインを含み、リラックスして眠気を防ぎたい人向けのものでごわ
Sound generation finished.
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|███████████████████████████████████| 72/72 [00:08<00:00,  8.48it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 241) to (256, 256) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The generated video is named ../付喪神ジェネレータ/results/2024_09_15_11.33.20/香るブラック君##output_black.mp4
using safetensor as default
3DMM Extraction for source image


Face Renderer:: 100%|█████████████████████████████████| 151/151 [00:17<00:00,  8.55it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The generated video is named ../付喪神ジェネレータ/results/2024_09_15_11.33.36/午後の紅茶さん##output_tea.mp4
Video generation finished.
0
responce_video_1 ../付喪神ジェネレータ/results/2024_09_15_11.33.20.mp4
responce_video_2 ../付喪神ジェネレータ/results/2024_09_15_11.33.36.mp4
